In [14]:
import pandas as pd
import numpy as np
import re
import json
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
import time

In [15]:
def split_coordinate(four_coords, divisions_longs, devision_lats, if_big_box):
    
    if if_big_box:
        [min_latitude, max_latitude, min_longitude, max_longitude] = [float(x) for x in four_coords.split(':')]
    else:
         [min_latitude, max_latitude, min_longitude, max_longitude] = four_coords

    longitude_step = (max_longitude - min_longitude) / divisions_longs
    latitude_step = (max_latitude - min_latitude) / devision_lats  # Typo: should be "divisions_lats"

    coord_boxes = []
    
    # Generate bounding boxes for each grid cell
    for i in range(divisions_longs):
        for j in range(devision_lats):
            box_min_lat = round(min_latitude + j * latitude_step, 5)
            box_max_lat = round(min_latitude + (j + 1) * latitude_step, 5)
            box_min_lon = round(min_longitude + i * longitude_step, 5)
            box_max_lon = round(min_longitude + (i + 1) * longitude_step, 5)

            # Store bounding box as a string in the format "min_lat:max_lat:min_lon:max_lon"
            box_str = f"{box_min_lat}:{box_max_lat}:{box_min_lon}:{box_max_lon}"
            coord_boxes.append(box_str)
    
    return coord_boxes

In [16]:
def vancouver_grid(head, divisions_longs, devision_lats):
    """
    Generates a grid of latitude-longitude bounding boxes within Vancouver's city boundary.

    Parameters:
    head (dict): Headers for the API request.
    divisions_longs (int): Number of divisions along the longitude (default is 15).
    devision_lats (int): Number of divisions along the latitude (default is 15).

    Returns:
    list: A list of strings representing bounding boxes in the format "min_lat:max_lat:min_lon:max_lon".
    """
    
    # API endpoint for Vancouver city boundary geo-coordinates
    van_geo_info_url = 'https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/city-boundary/records?limit=20'
    
    # Fetch geographical data from the API
    response = requests.get(van_geo_info_url, headers=head)
    geo_data = response.json()
    
    # Extract the city boundary coordinates
    contour = geo_data['results'][0]['geom']['geometry']['coordinates']

    # Flatten the list of coordinates and extract longitude and latitude values separately
    longitudes = [coord[0] for sublist in contour for coord in sublist]
    latitudes = [coord[1] for sublist in contour for coord in sublist]

    # Determine the minimum and maximum longitude and latitude values
    max_longitude = max(longitudes)
    min_longitude = min(longitudes)
    max_latitude = max(latitudes)
    min_latitude = min(latitudes)
    four_coords = [min_latitude, max_latitude, min_longitude, max_longitude]

    
    coord_boxes = split_coordinate(four_coords, divisions_longs, devision_lats, if_big_box = 0)
 
    
    return coord_boxes


In [17]:
def listing_count(head, coord_box):
    """
    Fetches the number of real estate listings within a specified coordinate box from Redfin.

    Parameters:
    head (dict): Headers for the HTTP request.
    coord_box (str): A string representing the bounding box in the format "min_lat:max_lat:min_lon:max_lon".

    Returns:
    tuple: (viewport_url, select_listing_count, total_listing_count)
        - viewport_url (str): The URL used for the request.
        - select_listing_count (int): The number of listings shown in the current viewport.
        - total_listing_count (int): The total number of listings available.
        - If no listings are found, returns 'no_listing'.
    """
    
    # Construct the URL for the given coordinate box
    viewport_url = f"https://www.redfin.ca/bc/vancouver/filter/viewport={coord_box}"
    
    # Send a GET request to fetch the webpage
    resp = requests.get(viewport_url, headers=head)

    # Raise an error if the request fails (non-200 status code)
    if resp.status_code != 200:
        raise Exception("Failing in webpage requests")
    
    # Parse the HTML response using BeautifulSoup
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Check if the page indicates no listings are available
    if soup.find('div', {'class': 'HomeViews reversePosition'}).find('h2'):
        return 'no_listing'
    
    # Extract the listing summary section
    listing_summary = soup.find('div', {'class': "homes summary reversePosition"})

    # Use regex to extract numeric values from the listing summary
    select_listing_count, total_listing_count = re.findall(r'\d{1,10}(?:,\d{1,10})*', listing_summary.text)
    
    # Convert extracted strings into integers, handling comma formatting
    select_listing_count, total_listing_count = int(select_listing_count), int(total_listing_count.replace(',', ''))
    
    return viewport_url, select_listing_count, total_listing_count


In [18]:
def crawling_redfin(head, viewport_url, page):
    """
    Crawls a specific page of real estate listings from Redfin within a given viewport.

    Parameters:
    head (dict): Headers for the HTTP request.
    viewport_url (str): Base URL for the listings search.
    page (int): Page number to crawl.

    Returns:
    list: A list of BeautifulSoup objects representing individual property listings.
    """
    
    # Construct the URL for the specified page number
    target_url = f"{viewport_url}/page-{page}"
    
    # Send a GET request to fetch the webpage
    resp = requests.get(target_url, headers=head)

    # Raise an error if the request fails (non-200 status code)
    if resp.status_code != 200:
        raise Exception("Failing in webpage requests")
    
    # Parse the HTML response using BeautifulSoup
    soup = BeautifulSoup(resp.text, 'html.parser')

    return soup

In [ ]:
def key_metric_extraction(soup, real_estate_info):
    """
    Extracts key real estate metrics from Redfin listing elements.

    Parameters:
    soup_boxes (list): A list of BeautifulSoup objects representing property listings.
    real_estate_info (dict): A dictionary to store extracted real estate information. 
                             The dictionary should have keys: 'address', 'zip_code', 'price', 
                             'bed', 'bath', 'sqr_footage', and 'property_link'.

    Returns:
    list: A list of indices where data extraction was incomplete.
    """
    
    incomplete_idx = []  # Stores indices of listings with missing data

    soup_boxes = soup.find_all("div", {"class": "HomeCardContainer"})

    for i, box in enumerate(soup_boxes):
        try:
            # Extract address (excluding last 23 characters, likely city/state info)
            address = box.find('address').text[:(-23)]
            real_estate_info['address'].append(address)
        except: 
            real_estate_info['address'].append(np.nan)
            incomplete_idx.append(i)

        try:
            # Extract ZIP code (last 7 characters of address text)
            zip_code = box.find('address').text[-7:]
            real_estate_info['zip_code'].append(zip_code)
        except: 
            real_estate_info['zip_code'].append(np.nan)
            incomplete_idx.append(i)        

        try:
            # Extract price
            price = box.find('span', {'class': 'bp-Homecard__Price--value'}).text
            real_estate_info['price'].append(price)
        except: 
            real_estate_info['price'].append(np.nan)
            incomplete_idx.append(i)

        try:
            # Extract number of bedrooms
            bed = box.find('span', {'class': 'bp-Homecard__Stats--beds text-nowrap'}).text
            real_estate_info['bed'].append(bed)
        except: 
            real_estate_info['bed'].append(np.nan)
            incomplete_idx.append(i)   

        try:
            # Extract number of bathrooms
            bath = box.find('span', {'class': 'bp-Homecard__Stats--baths text-nowrap'}).text
            real_estate_info['bath'].append(bath)
        except: 
            real_estate_info['bath'].append(np.nan)
            incomplete_idx.append(i)   

        try:
            # Extract square footage (locked stats section)
            sqr_footage = box.find('span', {'class': 'bp-Homecard__LockedStat--value'}).text
            real_estate_info['sqr_footage'].append(sqr_footage)
        except:
            real_estate_info['sqr_footage'].append(np.nan)
            incomplete_idx.append(i)   

        try:
            # Extract property link (prepend base URL)
            property_link = "https://www.redfin.com" + box.find("a").get('href')
            real_estate_info['property_link'].append(property_link)
        except:
            real_estate_info['property_link'].append(np.nan)
            incomplete_idx.append(i)

    return incomplete_idx


In [179]:
def calculate_min_pages(total_count, items_per_page):
    """
    Calculates the minimum number of pages required to display all items.

    Parameters:
    total_count (int): The total number of items to be displayed.
    items_per_page (int): The maximum number of items that can be displayed per page.

    Returns:
    int: The minimum number of pages required.
    """
    
    # Use integer division to determine the number of pages needed
    # Adding (items_per_page - 1) ensures proper rounding up
    return (total_count + items_per_page - 1) // items_per_page

In [180]:
def extracting_by_batch(head, batch_num, divisions_longs=15, devision_lats=15, splitted_big_box = 0):
    """
    Extracts real estate listing data from Redfin in batches using predefined coordinate grids.

    Parameters:
    head (dict): Headers for the HTTP requests.
    batch_num (int): The number of batches to divide the coordinate boxes into.
    test_batch_index (list, optional): The range of batch indices to process. Defaults to [0,1].

    Returns:
    tuple: (real_estate_info, missing_entries, big_coord_boxes)
        - real_estate_info (dict): A dictionary storing extracted real estate information.
        - missing_entries (dict): A dictionary tracking missing entries for incomplete pages.
        - big_coord_boxes (list): A list of coordinate boxes requiring further subdivision.
    """
    
    big_coord_boxes = []  # Stores coordinate boxes where select listing count < total listing count
    real_estate_info = defaultdict(list)  # Dictionary to store extracted real estate information
    missing_entries = defaultdict(list)  # Dictionary to track missing data entries

    # Generate the coordinate grid for Vancouver and split into batches
    if splitted_big_box:
        coord_boxes = splitted_big_box

    else: coord_boxes = vancouver_grid(head, divisions_longs, devision_lats)
    
    coord_box_batch = np.array_split(coord_boxes, batch_num)

    # Iterate over the specified batch range
    for i in range(len(coord_box_batch)):
        batch = coord_box_batch[i]

        # Process each coordinate box in the batch
        for coord_box in batch:
            listing_info = listing_count(head, coord_box)
            time.sleep(1)  # Prevent overwhelming the server

            # Skip if there are no listings in the area
            if listing_info == 'no_listing':
                print(f"Batch {i}-{coord_box} has no listings.")
                continue
            else:
                viewport_url, select_listing_count, total_listing_count = listing_info

                # If the selected listing count is less than the total, store the coordinate box for further subdivision
                if select_listing_count != total_listing_count:
                    big_coord_boxes.append(coord_box)
                    continue
                else:
                    # Calculate the number of pages to crawl based on listings per page (assumed 9 per page)
                    max_page = calculate_min_pages(select_listing_count, items_per_page=9)
                    missing = defaultdict(list)  # Tracks missing indices for this coordinate box

                    # Crawl and extract data for each page
                    for page in range(1, max_page):
                        soup_boxes = crawling_redfin(head, viewport_url, page)                        
                        incomplete_idx = key_metric_extraction(soup_boxes, real_estate_info)

                        # Store any missing data indices
                        if incomplete_idx:
                            missing[f'page_{page}'].append(incomplete_idx)

                        time.sleep(1)  # Prevent overwhelming the server

                    # Store missing entries for this coordinate box
                    missing_entries[coord_box].append(missing)

    return real_estate_info, missing_entries, big_coord_boxes

In [9]:
head = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
real_estate_info, missing_entries, big_coord_boxes = extracting_by_batch(head, batch_num=5, divisions_longs=6, devision_lats=6)

Batch 0-49.19835:49.21799:-123.22479:-123.19118 has no listings.
Batch 0-49.21799:49.23762:-123.22479:-123.19118 has no listings.
Batch 0-49.2769:49.29654:-123.22479:-123.19118 has no listings.
Batch 0-49.29654:49.31617:-123.22479:-123.19118 has no listings.
Batch 0-49.19835:49.21799:-123.19118:-123.15758 has no listings.
Batch 1-49.2769:49.29654:-123.19118:-123.15758 has no listings.
Batch 1-49.29654:49.31617:-123.19118:-123.15758 has no listings.
Batch 2-49.29654:49.31617:-123.15758:-123.12397 has no listings.
Batch 3-49.29654:49.31617:-123.12397:-123.09036 has no listings.
Batch 4-49.29654:49.31617:-123.09036:-123.05676 has no listings.
Batch 4-49.29654:49.31617:-123.05676:-123.02315 has no listings.


In [11]:
result = pd.DataFrame(real_estate_info)
result

,address,zip_code,price,bed,bath,sqr_footage,property_link
0,4091 W 34th Ave,V6N 2L6,"$2,788,000",5 beds,2 baths,"1,653",https://www.redfin.com/bc/vancouver/4091-W-34t...
1,4067 W 37th Ave,V6N 2W6,"$4,800,000",6 beds,4 baths,"4,189",https://www.redfin.com/bc/vancouver/4067-W-37t...
2,4056 W 36th Ave,V6N 2S9,"$6,750,000",4 beds,3 baths,"3,837",https://www.redfin.com/bc/vancouver/4056-W-36t...
3,3968 W 23rd Ave,V6S 1L2,"$3,388,000",4 beds,3.5 baths,"2,359",https://www.redfin.com/bc/vancouver/3968-W-23r...
4,4022 W 30th Ave,V6S 1X5,"$6,590,000",5 beds,5.5 baths,"4,190",https://www.redfin.com/bc/vancouver/4022-W-30t...
...,...,...,...,...,...,...,...
8322,2871 E Pender St,V5K 2C1,"$2,380,000",5 beds,2 baths,"1,613",https://www.redfin.com/bc/vancouver/2871-E-Pen...
8323,3635 Turner St #1,V5K 2K2,"$1,469,600",3 beds,3.5 baths,"1,336",https://www.redfin.com/bc/vancouver/3635-Turne...
8324,3615 Yale St,V5K 1C8,"$2,278,000",7 beds,5 baths,"2,930",https://www.redfin.com/bc/vancouver/3615-Yale-...
8325,2656 Adanac St,V5K 2M7,"$1,599,000",4 beds,1 bath,"1,900",https://www.redfin.com/bc/vancouver/2656-Adana...


In [14]:
result.to_csv("../data/vancouver_real_estate.csv")

In [ ]:
def extract_long_lat(head, link, pinpoints_info):
    resp = requests.get(link, headers=head)
    if resp.status_code != 200:
        raise Exception("Failing in webpage requests")
    else:
        soup = BeautifulSoup(resp.text, 'html.parser')

    info_list = soup.find_all('script', type='application/ld+json')
    
    for info in info_list:
        info = json.loads(info.string)
        try: 
            if info.get('name') == 'Redfin':
                continue
        except:
            
            try: 
                address = info[0].get('address').get('streetAddress')
                pinpoints_info['address'].append(address)
            except:
                pinpoints_info['address'].append(np.nan)
            try:
                lat = info[0].get('geo').get('latitude')
                pinpoints_info['latitude'].append(lat)
            except:
                pinpoints_info['latitude'].append(np.nan)
            try:
                long = info[0].get('geo').get('longitude')
                pinpoints_info['longitude'].append(long)
            except:
                pinpoints_info['longitude'].append(np.nan)
            try:
                region = info[0].get('address').get('addressLocality')
                pinpoints_info['region'].append(region)
            except:
                pinpoints_info['region'].append(np.nan)
            try:
                zip_code = info[0].get('address').get('postalCode')
                pinpoints_info['zip_code'].append(zip_code)
            except:
                pinpoints_info['zip_code'].append(np.nan)
            try:
                square_footage = info[0].get('floorSize').get('value')
                pinpoints_info['square_footage'].append(square_footage)
            except:
                pinpoints_info['square_footage'].append(np.nan)
            try:
                bedroom = info[0].get('numberOfRooms')
                pinpoints_info['bedroom'].append(bedroom)
            except:
                pinpoints_info['bedroom'].append(np.nan)
            try:
                url = info[0].get('url')
                pinpoints_info['url'].append(url)
            except:
                pinpoints_info['url'].append(np.nan)
            try:
                price = info[1].get('offers').get('price')
                pinpoints_info['price'].append(price)
            except:
                pinpoints_info['price'].append(np.nan)

    return pinpoints_info

In [218]:
from bs4 import BeautifulSoup
import requests, json
import numpy as np

def extract_long_lat(head, link, pinpoints_info):
    resp = requests.get(link, headers=head)
    if resp.status_code != 200:
        raise Exception("Failing in webpage requests")
    
    soup = BeautifulSoup(resp.text, 'html.parser')
    scripts = soup.find_all('script', type='application/ld+json')

    listings = {}

    for script in scripts:
        try:
            data = json.loads(script.string)
        except Exception:
            continue

        items = data if isinstance(data, list) else [data]

        for item in items:
            if item.get('name') == 'Redfin':
                continue  # Skip metadata

            # Use URL as key, or fallback to address
            url = item.get('url') or item.get('mainEntityOfPage', {}).get('@id')
            address = item.get('address', {}).get('streetAddress')
            key = url or address
            if not key:
                continue

            if key not in listings:
                listings[key] = {
                    'address': np.nan,
                    'latitude': np.nan,
                    'longitude': np.nan,
                    'region': np.nan,
                    'zip_code': np.nan,
                    'square_footage': np.nan,
                    'bedroom': np.nan,
                    'url': key,
                    'price': np.nan
                }

            # Fill in whatever data is present
            if item.get('address'):
                listings[key]['address'] = item['address'].get('streetAddress', listings[key]['address'])
                listings[key]['region'] = item['address'].get('addressLocality', listings[key]['region'])
                listings[key]['zip_code'] = item['address'].get('postalCode', listings[key]['zip_code'])

            if item.get('geo'):
                listings[key]['latitude'] = item['geo'].get('latitude', listings[key]['latitude'])
                listings[key]['longitude'] = item['geo'].get('longitude', listings[key]['longitude'])

            if item.get('floorSize'):
                listings[key]['square_footage'] = item['floorSize'].get('value', listings[key]['square_footage'])

            if 'numberOfRooms' in item:
                listings[key]['bedroom'] = item['numberOfRooms']

            if 'url' in item:
                listings[key]['url'] = item['url']

            if 'offers' in item and isinstance(item['offers'], dict):
                listings[key]['price'] = item['offers'].get('price', listings[key]['price'])

    # Now append each full listing once
    for listing in listings.values():
        for k, v in listing.items():
            pinpoints_info[k].append(v)

    return pinpoints_info


In [237]:
def brut_force_extraction_long_lat(head, divisions_longs=20, devision_lats=20, test=[300,301]):

    pinpoints_info = defaultdict(list)  
    coord_boxes = vancouver_grid(head, divisions_longs, devision_lats)
    
    for i in range(test[0], test[1]):
        coord_box = coord_boxes[i]
        viewport_url = f"https://www.redfin.ca/bc/vancouver/filter/viewport={coord_box}/page-2"
        extract_long_lat(head, viewport_url, pinpoints_info)
        time.sleep(1)

    return pinpoints_info
        

In [233]:
pd.DataFrame(p)

,address,latitude,longitude,region,zip_code,square_footage,bedroom,url,price
0,340 Woodland Dr,49.281474,-123.072170,Salt Spring,V8K 1J6,4045,3,https://www.redfin.ca/bc/vancouver/340-Woodlan...,1275000
1,3092 W 38th Ave,49.237353,-123.174353,Vancouver,V6N 2X3,3295,5,https://www.redfin.ca/bc/vancouver/3092-W-38th...,3498000
2,219 E Georgia St #303,49.278812,-123.098928,Vancouver,V6A 1Z6,495,1,https://www.redfin.ca/bc/vancouver/219-E-Georg...,519000
3,231 E Pender St #404,49.280468,-123.098698,Vancouver,V6A 0G3,505,0,https://www.redfin.ca/bc/vancouver/231-E-Pende...,459000
4,1119 Shavington St,49.313197,-123.043746,North Vancouver,V7L 1K7,2552,4,https://www.redfin.ca/bc/north-vancouver/1119-...,1799000
5,125 Milross Ave #508,49.276283,-123.101662,Vancouver,V6A 0A1,750,2,https://www.redfin.ca/bc/vancouver/125-Milross...,850000
6,5248 Inverness St,49.237316,-123.079993,Vancouver,V5W 3N8,2468,6,https://www.redfin.ca/bc/vancouver/5248-Invern...,2348000
7,1189 Melville St #4202,49.287804,-123.123642,Vancouver,V6E 4T8,882,2,https://www.redfin.ca/bc/vancouver/1189-Melvil...,1499000
8,2248 E 48th Ave #1,49.226041,-123.061020,Vancouver,V5P 1S1,1560,4,https://www.redfin.ca/bc/vancouver/2248-E-48th...,1638000
9,8080 Cambie Rd #525,49.184533,-123.135253,Richmond,V6X 0C1,715,1,https://www.redfin.ca/bc/richmond/8080-Cambie-...,668000
